In [ ]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
import numpy as np
from numpy import random

import nltk
import gensim
from gensim import corpora, models, similarities
nltk.download('punkt')
nltk.download('stopwords')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import re

In [ ]:
### load the meta data - USING DOWNLOADED DATASET

data = []
with gzip.open('reviews_Video_Games_5.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[0])

In [ ]:
# convert list into pandas dataframe

df = pd.DataFrame.from_dict(data)

print(len(df))
df.columns

In [ ]:
df.isna().sum()

In [ ]:
x = df.loc[:,'reviewText']
y = df.loc[:,'overall']

In [ ]:
#PREPROCESSING OUTPUTS
y_new = []
for i in range(y.size):
    if y[i] < 4:
        y_new.append(0) #bad
    else:
        y_new.append(1) #good

print(y_new.count(1))
print(y_new.count(0))
print(y_new.count(1)/y_new.count(0))

In [ ]:
#PREPROCESSING INPUTS
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = nltk.corpus.stopwords.words('english')

for i in range(x.size):
    x[i] = x[i].lower()
    x[i] = REPLACE_BY_SPACE_RE.sub(' ', x[i]) # replace REPLACE_BY_SPACE_RE symbols by space in text
    x[i] = BAD_SYMBOLS_RE.sub('', x[i]) # delete symbols which are in BAD_SYMBOLS_RE from text
    x[i] = ' '.join(word for word in x[i].split() if word not in STOPWORDS)

In [ ]:
# DATAFRAME
list_of_data = list(zip(x, y_new))
df_processed = pd.DataFrame(list_of_data, columns=['input','output'])

In [ ]:
X = df_processed.input
y = df_processed.output
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.1, random_state = 1)

In [ ]:
#WORD2VEC
# https://radimrehurek.com/gensim/models/word2vec.html
corpus = []
for i in range(x.size):
    corpus.append(x[i])

all_words = [nltk.word_tokenize(sent) for sent in corpus]

# model = Word2Vec(sentences=common_texts, vector_size=100, window=5, min_count=1, workers=4)
model = Word2Vec(sentences=all_words, min_count=2, workers=4)
model.save("word2vec.model")

In [ ]:
#SAVING Store just the words + their trained embeddings.
from gensim.models import KeyedVectors
word_vectors = word2vec.wv
word_vectors.save("word2vec.wordvectors")

In [ ]:
## Not exactly sure how to model it... or if I modeled it correctly at the #WORD2VEC part